In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

df = pd.read_csv("/kaggle/input/full-filled-brain-stroke-dataset/full_data.csv")
df_eval = pd.read_csv("/kaggle/input/full-filled-brain-stroke-dataset/full_filled_stroke_data (1).csv")

/kaggle/input/full-filled-brain-stroke-dataset/full_data.csv
/kaggle/input/full-filled-brain-stroke-dataset/full_filled_stroke_data (1).csv


In [2]:
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [3]:
y_train = df.pop("stroke")
y_eval = df_eval.pop("stroke")

In [4]:
df.loc[df['gender']=='Male','gender']=1
df.loc[df['gender']=='Female','gender']=0
df.loc[df['ever_married']=='Yes','ever_married']=1
df.loc[df['ever_married']=='No','ever_married']=0
df.loc[df['work_type']=='Private','work_type']=0
df.loc[df['work_type']=='Self-employed','work_type']=1
df.loc[df['work_type']=='Govt_job','work_type']=2
df.loc[df['work_type']=='children','work_type']=3
df.loc[df['Residence_type']=='Urban','Residence_type']=1
df.loc[df['Residence_type']=='Rural','Residence_type']=0
df.loc[df['smoking_status']=='formerly smoked','smoking_status']=0
df.loc[df['smoking_status']=='never smoked','smoking_status']=1
df.loc[df['smoking_status']=='smokes','smoking_status']=2
df.loc[df['smoking_status']=='Unknown','smoking_status']=3
df_eval.loc[df_eval['gender']=='Male','gender']=1
df_eval.loc[df_eval['gender']=='Female','gender']=0
df_eval.loc[df_eval['ever_married']=='Yes','ever_married']=1
df_eval.loc[df_eval['ever_married']=='No','ever_married']=0
df_eval.loc[df_eval['work_type']=='Private','work_type']=0
df_eval.loc[df_eval['work_type']=='Self-employed','work_type']=1
df_eval.loc[df_eval['work_type']=='Govt_job','work_type']=2
df_eval.loc[df_eval['work_type']=='children','work_type']=3
df_eval.loc[df_eval['Residence_type']=='Urban','Residence_type']=1
df_eval.loc[df_eval['Residence_type']=='Rural','Residence_type']=0
df_eval.loc[df_eval['smoking_status']=='formerly smoked','smoking_status']=0
df_eval.loc[df_eval['smoking_status']=='never smoked','smoking_status']=1
df_eval.loc[df_eval['smoking_status']=='smokes','smoking_status']=2
df_eval.loc[df_eval['smoking_status']=='Unknown','smoking_status']=3


In [5]:
df['age'] = df['age']/100
df['avg_glucose_level'] = df['avg_glucose_level']/300
df['bmi'] = df['bmi']/50

In [6]:
dfnew = df.to_numpy()
dfevalnew = df_eval.to_numpy()
ytrainnew = y_train.to_numpy() 
yevalnew = y_eval.to_numpy()
X = dfnew.T
X_eval = dfevalnew.T
n_x = X.shape[0]
m = X.shape[1]
X=np.array(X,dtype=np.float64)
X_eval=np.array(X_eval,dtype=np.float64)
n_h = 15

In [7]:
W1 = np.random.randn(n_h,n_x)*0.01
b1 = np.zeros((n_h,1))
W2 = np.random.randn(1,n_h)*0.01
b2 = np.zeros((1,1))

In [8]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [9]:
def relu(z):
    return np.maximum(0,z)


In [10]:
def costfunc(X,W1, b1, W2, b2, y):
    Z1= np.dot(W1,X) + b1
    A1=relu(Z1)
    Z2= np.dot(W2,A1) + b2
    A2 = sigmoid(Z2);
    cost = np.sum(-y*np.log(A2) - (1-y)*np.log(1-A2))/m 
    dZ2 = A2-y
    db2 = np.sum(dZ2, axis=1,keepdims=True)/m
    dW2 = np.dot(dZ2,A1.T)/m
    dZ1 = np.multiply(np.dot(W2.T,dZ2),1-np.power(A1,2))
    dW1 = np.dot(dZ1, X.T)/m
    db1 = np.sum(dZ1,axis=1,keepdims=True)/m
    return cost,dW1,db1,dW2,db2

In [11]:
costfunc(X,W1, b1, W2, b2, ytrainnew)

(0.6932551011841225,
 array([[ 2.38540185e-04,  2.34553887e-04,  4.45686165e-05,
          2.32553810e-05,  3.64970002e-04,  4.93009925e-04,
          2.90438648e-04,  1.97845635e-04,  3.26141465e-04,
          9.32764113e-04],
        [-1.70226012e-03, -1.67390999e-03, -3.18052861e-04,
         -1.65928276e-04, -2.60493219e-03, -3.51469058e-03,
         -2.07257775e-03, -1.41169786e-03, -2.32723184e-03,
         -6.65333843e-03],
        [ 2.65536940e-03,  2.61036655e-03,  4.95978756e-04,
          2.58767299e-04,  4.06211314e-03,  5.48410420e-03,
          3.23281601e-03,  2.20171353e-03,  3.62948832e-03,
          1.03782345e-02],
        [-5.89347852e-04, -5.79354926e-04, -1.10077616e-04,
         -5.74318006e-05, -9.01551503e-04, -1.21714289e-03,
         -7.17501709e-04, -4.88659899e-04, -8.05548640e-04,
         -2.30343000e-03],
        [-6.64844114e-04, -6.53589675e-04, -1.24176888e-04,
         -6.47886152e-05, -1.01707505e-03, -1.37301052e-03,
         -8.09453247e-04, -5.51

In [12]:
def gradient_descent(X,W1,b1, W2, b2, m,ytrainnew, learning_rate, learning_rate2):
    for i in range(1000):
        cost,dW1,db1,dW2,db2=costfunc(X,W1, b1, W2, b2, ytrainnew)
        print(i, cost)
        W1 = W1 - learning_rate*dW1
        b1 = b1 - learning_rate*db1
        W2 = W2 - learning_rate2*dW2
        b2 = b2 - learning_rate2*db2
        

In [13]:
gradient_descent(X,W1,b1, W2, b2, m,ytrainnew,0.1,9)

0 0.6932551011841225
1 0.21936140723661773
2 0.21040425211350666
3 0.20446557606255564
4 0.2010029406849276
5 0.19923554960084888
6 0.1984372972350827
7 0.19811036951169014
8 0.19798467037960238
9 0.19793708886465236
10 0.19791795421681585
11 0.19790876971272905
12 0.19790298451718624
13 0.19789834794696856
14 0.19789409282795756
15 0.19788996076462576
16 0.19788586351727444
17 0.19788176557905646
18 0.1978776532641482
19 0.1978735141279794
20 0.19786933452179442
21 0.19786510276145558
22 0.19786081231113187
23 0.19785645012691933
24 0.1978520051458455
25 0.19784747487536267
26 0.19784285177241556
27 0.19783811515140098
28 0.19783325397885598
29 0.1978282670693685
30 0.19782314129815526
31 0.19781785983357894
32 0.19781241154008766
33 0.19780677958413556
34 0.19780095545835089
35 0.19779492461419693
36 0.19778867348401635
37 0.19778219070339853
38 0.1977754564787779
39 0.19776845444268007
40 0.19776117296800488
41 0.1977535978326179
42 0.19774571210302688
43 0.19773750489830927
44 0.19

In [14]:
def predict(X,W1,b1,W2,b2,yevalnew):
    print(yevalnew)
    Z1 = np.dot(W1,X)+b1
    m = X.shape[1]
    A1 = relu(Z1)
    Z2 = np.dot(W2,A1)+b2
    A2 = sigmoid(Z2)
    predictor = np.zeros(m)
    t=f=0
    for i in range(m):
        if(A2[0][i]>0.5):
            predictor[i]=1
        if(predictor[i]==yevalnew[i]):
            t+=1
        else:
            f+=1
    print(t, f)
    print(predictor)
    print("accuracy: ", (t/(t+f))*100)

In [15]:
predict(X_eval,W1,b1,W2,b2,yevalnew)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
40 161
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
